In [ ]:
### HYPERPARAMETERS ###
MA_WINDOW = 253
THRESHOLD = 0.015
### HYPERPARAMETERS ###

In [ ]:
import os
import pandas as pd
import yfinance as yf


def reindex_and_fill(
    data,
    first_date,
    last_date,
    freq,
):
    date_range = pd.date_range(first_date, last_date, freq=freq)
    new_data = data.reindex(date_range)
    new_data = new_data.ffill()
    new_data = new_data.bfill()
    return new_data


def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0] for col in df.columns.values]
    return df


def ensure_symbol_data(symbol, data_dir="data"):
    os.makedirs(data_dir, exist_ok=True)
    filename = symbol.replace("^", "").lower() + ".csv"
    filepath = os.path.join(data_dir, filename)
    today = pd.Timestamp.today().normalize()  # midnight of today for comparison

    # Check if file exists AND its latest date >= current date
    if os.path.exists(filepath):
        existing = pd.read_csv(filepath, parse_dates=["Date"])
        if not existing.empty:
            last_date = existing["Date"].max().normalize()
            # If already have today's data, just return it!
            if last_date >= today:
                existing.set_index("Date", inplace=True)
                return existing

    # Otherwise, download the latest data
    data = yf.download(
        symbol,
        period="max",
        interval="1d",
        auto_adjust=False,
        actions=False,
        progress=False,
    )
    data = flatten_columns(data)
    data.index.name = "Date"
    data.reset_index(inplace=True)

    # Merge with existing data if present
    if os.path.exists(filepath):
        existing = pd.read_csv(filepath, parse_dates=["Date"])
        combined = pd.concat([existing, data], ignore_index=True)
        combined.drop_duplicates(subset=["Date"], keep="last", inplace=True)
        combined.sort_values("Date", inplace=True)
    else:
        combined = data

    # Ensure columns are in a fixed order and only the exact columns from Yahoo
    columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
    combined = combined[[col for col in columns if col in combined.columns]]

    combined.to_csv(filepath, index=False)
    # Load the dataframe and set Date as a datetime index!
    df = pd.read_csv(filepath, parse_dates=["Date"], index_col="Date")
    return df


gspc = ensure_symbol_data("^GSPC")
gspc = reindex_and_fill(gspc, min(gspc.index), max(gspc.index), "D")
gspc = gspc.rename(columns=lambda x: x.replace(" ", "_"))
print(gspc.tail())

In [ ]:
# Test with simulated data
# simulated data deviates slightly from the s&p500 data from yahoo

# df = pd.read_excel(
#     "/home/max/Git/zgea-github/clean_data/etfs.xlsx",
#     index_col=0,  # first column becomes index
#     parse_dates=True,
# )

# gspc = df[["1x_sp500_eu"]].rename(columns={"1x_sp500_eu": "Adj_Close"})
# print(gspc.tail())

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, timezone
import pytz

# Check during US trading hours
eastern = pytz.timezone("US/Eastern")
now_et = datetime.now(eastern)
is_weekday = now_et.weekday() < 5
after_open = now_et.hour > 9 or (now_et.hour == 9 and now_et.minute >= 30)
before_close = now_et.hour < 16
if not (is_weekday and after_open and before_close):
    raise RuntimeError(
        "US stock market is closed. Try again during regular trading hours (Mon-Fri, 9:30-16:00 ET)."
    )

# Use timezone-aware UTC "now", rounded down to minute
now_utc = datetime.now(timezone.utc).replace(second=0, microsecond=0)
end = now_utc
start = end - timedelta(minutes=30)
print(f"Fetching SPX data from {start} to {end} UTC")

data = yf.download(
    "^GSPC",
    start=start,
    end=end,
    interval="1m",
    auto_adjust=False,
    actions=False,
    progress=False,
)
data = flatten_columns(data)
data.index.name = "Date"

if data.empty or "Adj Close" not in data.columns:
    raise RuntimeError(
        "No SPX 1m price data returned. Check that the market is open and try later (Yahoo may delay 1m data)."
    )

s = pd.to_numeric(data["Adj Close"], errors="coerce")
last_valid_idx = s.last_valid_index()
last_valid_val = s.loc[last_valid_idx]

# Update or append in gspc (assume DateTimeIndex and 'Adj_Close' column)
today_str = now_utc.strftime("%Y-%m-%d")
last_date_str = gspc.index[-1].strftime("%Y-%m-%d")

if last_date_str == today_str:
    gspc.iloc[-1, gspc.columns.get_loc("Adj_Close")] = last_valid_val
else:
    next_date = gspc.index[-1] + pd.Timedelta(days=1)
    # If your gspc has more than one column, fill the rest with None:
    new_row = [None] * (gspc.shape[1] - 1) + [last_valid_val]
    gspc.loc[next_date] = new_row


In [ ]:
# Calculate the SMA
gspc["SMA"] = gspc["Adj_Close"].rolling(window=MA_WINDOW).mean()
gspc = gspc.dropna(subset=["SMA"])

# Calculate threshold bounds
gspc["upp"] = gspc["SMA"] * (1 + THRESHOLD)
gspc["low"] = gspc["SMA"] * (1 - THRESHOLD)

assert not gspc.isnull().values.any()

In [ ]:
gspc["signal"] = None
invested = False

for idx, row in enumerate(gspc.itertuples(), 0):
    if not invested and row.Adj_Close >= row.upp:
        gspc.iat[idx, gspc.columns.get_loc("signal")] = "BUY"
        invested = True
    elif invested and row.Adj_Close < row.low:
        gspc.iat[idx, gspc.columns.get_loc("signal")] = "SELL"
        invested = False

In [ ]:
import plotly.graph_objects as go
import pandas as pd

end = gspc.index.max()
start = end - pd.DateOffset(months=9)
data = gspc.loc[start:]
# data = gspc

# 1. Main traces: Adj_Close and SMA
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data.index,
        y=data["Adj_Close"],
        mode="lines",
        name="Adj Close",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=data.index, y=data["SMA"], mode="lines", name="SMA", line=dict(color="orange")
    )
)

# 2. Fill area between 'upp' and 'low'
fig.add_traces(
    [
        go.Scatter(
            x=data.index,
            y=data["upp"],
            mode="lines",
            line=dict(width=0),
            showlegend=False,
            hoverinfo="skip",
        ),
        go.Scatter(
            x=data.index,
            y=data["low"],
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(200,200,255,0.3)",
            line=dict(width=0),
            name="Threshold Band",
            showlegend=True,
            hoverinfo="skip",
        ),
    ]
)

# 3. Buy and sell markers
buy_points = data.loc[data.signal == "BUY"]
sell_points = data.loc[data.signal == "SELL"]

fig.add_trace(
    go.Scatter(
        x=buy_points.index,
        y=buy_points["Adj_Close"],
        mode="markers",
        marker=dict(symbol="triangle-up", color="green", size=12),
        name="Buy Signal",
    )
)

fig.add_trace(
    go.Scatter(
        x=sell_points.index,
        y=sell_points["Adj_Close"],
        mode="markers",
        marker=dict(symbol="triangle-down", color="red", size=12),
        name="Sell Signal",
    )
)

fig.update_layout(
    title="S&P 500 Adj Close with SMA, Threshold Area, Buy/Sell Signals",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1, xanchor="right", x=1),
)

fig.show()
